# SVD (Singular Value Decomposition) - 행렬 분해 기반 추천 모델

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# 내장 데이터셋 (MovieLens 100K)
data = Dataset.load_builtin('ml-100k')

# SVD 모델 생성
model = SVD()

# 교차 검증을 통한 성능 평가
cross_validate(model, data, cv=5, verbose=True)

# NCF (Neural Collaborative Filtering)

행렬 분해(Matrix Factorization, MF)를 딥러닝 기반으로 확장한 추천 모델

MF는
𝑅=
𝑈
⋅
𝑉
𝑇
R=U⋅V
T형태로 분해하지만,
NCF는 이를 신경망을 통해 학습

사용자 & 아이템을 임베딩 벡터로 변환 후 신경망을 통해 추천을 수행


MF와 MLP를 합친 NeuMF가 가장 강력한 성능을 보임


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# 사용자 & 아이템 수 정의 (예제)
num_users = 1000
num_items = 1000
embedding_dim = 32  # 임베딩 차원

# 입력 레이어
user_input = Input(shape=(1,), name="user_input")
item_input = Input(shape=(1,), name="item_input")

# 임베딩 레이어
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim)(item_input)

# Flatten (차원 축소)
user_vec = Flatten()(user_embedding)
item_vec = Flatten()(item_embedding)

# MLP 기반 추천 모델
concat = Concatenate()([user_vec, item_vec])
dense1 = Dense(128, activation='relu')(concat)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(32, activation='relu')(dense2)
output = Dense(1, activation='sigmoid')(dense3)  # 평점 예측

# 모델 정의
ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
ncf_model.summary()


# AutoEncoder 기반 추천 시스템

AutoEncoder를 사용해 사용자-아이템 행렬을 압축(차원 축소)하여 추천

기존 MF 모델보다 비선형적인 특징까지 학습 가능

희소 행렬을 압축하여 특징을 추출

기존 MF보다 비선형적인 패턴을 학습 가능

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# AutoEncoder 모델 정의
class AutoEncoder(nn.Module):
    def __init__(self, num_items, hidden_dim=64):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(num_items, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 32),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_items),
            nn.Sigmoid()  # 복원된 사용자-아이템 행렬
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 사용자-아이템 행렬 크기 정의
num_users, num_items = 1000, 1000
model = AutoEncoder(num_items)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 가짜 사용자-아이템 행렬 (랜덤 데이터)
input_data = torch.rand((num_users, num_items))

# 학습
for epoch in range(10):
    optimizer.zero_grad()
    output = model(input_data)
    loss = criterion(output, input_data)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")


# Graph-based 추천 시스템 (GNN 기반 추천)

추천 시스템을 그래프(Graph) 형태로 모델링하여 추천을 수행

사용자-아이템 관계를 이웃(Neighbor)으로 간주

사용자-아이템 관계를 그래프로 모델링하여 추천을 수행

협업 필터링보다 더 복잡한 관계(이웃 사용자, 아이템 간 유사성 등)를 학습 가능

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
import dgl.function as fn

# 그래프 신경망 모델 (GNN)
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super(GCN, self).__init__()
        self.layer1 = nn.Linear(in_feats, hidden_feats)
        self.layer2 = nn.Linear(hidden_feats, out_feats)

    def forward(self, graph, features):
        h = torch.relu(self.layer1(features))
        graph.ndata['h'] = h
        graph.update_all(fn.copy_u('h', 'm'), fn.mean('m', 'h'))
        h = graph.ndata.pop('h')
        h = self.layer2(h)
        return h

# 사용자-아이템 그래프 생성 (임의 데이터)
num_users, num_items = 100, 100
edges_src = torch.randint(0, num_users, (500,))
edges_dst = torch.randint(0, num_items, (500,)) + num_users

graph = dgl.graph((edges_src, edges_dst))
features = torch.rand((num_users + num_items, 32))

# 모델 정의
model = GCN(32, 64, 32)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# 학습 과정
for epoch in range(10):
    optimizer.zero_grad()
    output = model(graph, features)
    loss = loss_fn(output, features)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")


# 시퀀스 데이터 기반 추천 시스템 (Sequential Recommendation)

사용자의 과거 행동(클릭, 시청, 구매 등)을 시계열 데이터로 보고, 다음 행동을 예측하는 추천

단순한 협업 필터링보다 연속적인 사용자 패턴을 학습할 수 있음

Spotify, YouTube 추천: 이전에 들었던 노래/영상 기반으로 다음 곡 추천

Netflix, Disney+ 추천: 시청 패턴을 기반으로 추천

In [ ]:
import torch
import torch.nn as nn

# RNN 기반 추천 모델
class RNNRecSys(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRecSys, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝 출력 사용
        return out

# 입력: 사용자 행동 시퀀스 (예: 최근 본 영화들)
model = RNNRecSys(input_dim=32, hidden_dim=64, output_dim=10)


# Seq2Seq 모델을 추천 시스템에 적용하는 경우

**"입력 시퀀스를 다음 추천 아이템 시퀀스로 변환"**하는 문제에 적용 가능

"이전에 본 영화 → 다음에 볼 가능성이 높은 영화 시퀀스 추천"

"유저의 행동 패턴을 시퀀스로 보고, 다음 클릭할 아이템을 예측"

In [ ]:
class Seq2SeqRecSys(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Seq2SeqRecSys, self).__init__()
        self.encoder = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (h, c) = self.encoder(x)
        out, _ = self.decoder(x, (h, c))
        return self.fc(out[:, -1, :])

model = Seq2SeqRecSys(input_dim=32, hidden_dim=64, output_dim=10)


#  Attention 기반 추천 시스템 (Attention Mechanism in RecSys)

Attention은 사용자의 관심도(Attention Score)를 학습하여 더 적절한 아이템을 추천할 수 있도록 도움

TikTok, Instagram Reels 추천: 유저가 특정 스타일의 영상에 더 오래 머물면 해당 스타일에 높은 Attention을 부여

E-commerce (아마존, 쿠팡): 사용자가 집중했던 상품의 유사한 상품 추천

In [ ]:
class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super(AttentionLayer, self).__init__()
        self.attn = nn.Linear(input_dim, 1)

    def forward(self, x):
        attn_weights = torch.softmax(self.attn(x), dim=1)
        return torch.sum(attn_weights * x, dim=1)

# 사용자의 과거 행동 시퀀스를 Attention을 통해 추천
attention_layer = AttentionLayer(input_dim=32)


# Transformer가 추천 시스템에서 쓰이는 경우

BERT4Rec (Amazon 추천 시스템): BERT 모델을 활용해 순서를 무작위로 바꿔도 추천이 가능하도록 학습

SASRec (Self-Attention 기반 추천): LSTM보다 더 강력한 추천 성능

GPT 기반 추천 시스템: 대화형 추천 (예: ChatGPT 기반 추천)

In [ ]:
class SASRec(nn.Module):
    def __init__(self, num_items, embedding_dim):
        super(SASRec, self).__init__()
        self.embedding = nn.Embedding(num_items, embedding_dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=2),
            num_layers=2
        )
        self.fc = nn.Linear(embedding_dim, num_items)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        return self.fc(x[:, -1, :])

model = SASRec(num_items=1000, embedding_dim=32)


# 이미지 기반 추천

텍스트+이미지를 결합한 추천 모델이 많이 사용

Pinterest, 네이버 쇼핑 → "유사한 스타일의 패션 아이템 추천"

TikTok, YouTube → "영상 내용을 분석하여 비슷한 영상 추천"

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

image = torch.rand((3, 224, 224))  # 임의의 이미지 데이터
text = ["Nike 신발", "Adidas 신발", "Puma 신발"]
inputs = processor(images=image, text=text, return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image
recommended_index = logits_per_image.argmax()
print(f"추천 아이템: {text[recommended_index]}")
